In [ ]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 2 - 2.3 Segmentação de Negócio

### Geração de Massa

In [ ]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'solicitante.saldoMedio':[0],
                             'solicitante.rendaLiquidaPicpay':[10000],
                             'solicitante.flagPrincipalidadeAll':[0]
                            })

In [ ]:
#Criação das massas com casos forçados
#Digito CPF Concomitante CP
massa_segmentacaoNegocio = sherlock.Gera_Massa(base_entrada)
massa_segmentacaoNegocio.gera_cenario('solicitante.saldoMedio',[-99999,0,99999,100000,100001,39999,40000,40001])
massa_segmentacaoNegocio.gera_cenario('solicitante.rendaLiquidaPicpay',[-99999,11124.15,11124.16,11124.17,3442.36,3442.37,3442.38])
massa_segmentacaoNegocio.gera_cenario('solicitante.flagPrincipalidadeAll',[1,0])

massa_segmentacaoNegocio.get_massa_final()

massa_segmentacaoNegocio.set_bom_arqv()
massa_segmentacaoNegocio.converte_massa()

### Execução Modular e Geração de CSV

In [ ]:
listaResultado = rp.execucaoModular(massa_segmentacaoNegocio.get_massa_json(),rp.SegmentacaoNegocio)

##### Valida se todas condições estão sendo abordadas

In [ ]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['segmentacaoNegocio'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [ ]:
# Renomeação das colunas para o cabeçalho do RMA
massa_segmentacaoNegocio.rename_cabecalho()

In [ ]:
massa_segmentacaoNegocio.atribui_expected(listaResultado,"payloadHomol.intermediarias.segmentacaoNegocio","segmentacaoNegocio")
massa_segmentacaoNegocio.get_massa_dm()

### Conversão da massa para CSV

In [ ]:
massa_segmentacaoNegocio.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaSegmentacaoNegocio.csv',index=False)